In [ ]:
!pip install customtkinter

In [21]:
import customtkinter as ctk
from tkinter import messagebox
from tensorflow.keras.models import load_model
import pickle
import numpy as np
from nltk.tokenize import TweetTokenizer 

# Load model
model = load_model("CONV1D_LSTM_31acc.keras")  

# Load vocab dictionary
vocab = None
with open("vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

emoji_map = {
    0: "❤",
    1: "😍",
    2: "😂",
    3: "💕",
    4: "🔥",
    5: "😊",
    6: "😎",
    7: "✨",
    8: "💙",
    9: "😘",
    10: "📷",
    11: "🇺🇸",
    12: "☀",
    13: "💜",
    14: "😉",
    15: "💯",
    16: "😁",
    17: "🎄",
    18: "📸",
    19: "😜"
}


def tokenizeText(text): 
    tknzr = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True) 
    return tknzr.tokenize(text.lower())

def get_emoji_from_text(text):
    text = text.lower() 
     
    maxLen=40 
    wordVec = np.zeros(maxLen)
    words = tokenizeText(text)
    for j in range(min(len(words),maxLen)):
        if(vocab.get(words[j])) : wordVec[j]=vocab[words[j]]
    x = np.array([wordVec])
    # print(x)

    Y_predicted = model.predict(x)[0] 

    Y = [(Y_predicted[i],i) for i in range(len(Y_predicted))] 
    Y.sort(key=lambda a:-a[0]) 
    top3 = [it[1] for it in Y[0:3]]

    result = "" 
    for y in top3: 
        result += emoji_map[y]
        
    return result

def analyze_text():
    user_text = text_box.get("0.0", "end").strip()
    if not user_text:
        messagebox.showwarning("Empty Input", "Please type some text first!")
        return
    emoji = get_emoji_from_text(user_text)
    emoji_label.configure(text=emoji)

# --- App Setup ---
ctk.set_appearance_mode("dark")        # Modes: "light" or "dark"
ctk.set_default_color_theme("blue")    # Themes: "blue", "green", "dark-blue"

app = ctk.CTk()
app.title("Emotion → Emoji")
app.geometry("420x400")
app.resizable(False, False)

# --- Title ---
title_label = ctk.CTkLabel(
    app, text="💬 Emotion → Emoji 😊",
    font=ctk.CTkFont(size=22, weight="bold")
)
title_label.pack(pady=(25, 10))

# --- Input box ---
text_box = ctk.CTkTextbox(app, width=320, height=100, corner_radius=10)
text_box.pack(pady=10)

# --- Analyze button ---
analyze_button = ctk.CTkButton(
    app, text="Analyze Emotion", 
    command=analyze_text, 
    width=200, height=40, 
    corner_radius=20, font=ctk.CTkFont(size=15, weight="bold")
)
analyze_button.pack(pady=15)

# --- Result label (emoji display) ---
emoji_label = ctk.CTkLabel(
    app, text="✨", font=ctk.CTkFont(size=50, weight="bold")
)
emoji_label.pack(pady=20)

# --- Footer ---
footer_label = ctk.CTkLabel(
    app, text="Type how you feel above 💭", 
    font=ctk.CTkFont(size=12, slant="italic")
)
footer_label.pack(side="bottom", pady=10)

# --- Run app ---
app.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
